In [1]:
pip install langgraph

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langgraph.types import interrupt, Command
from typing import Annotated
from operator import add
from langgraph.checkpoint.memory import MemorySaver
import uuid


class workflowState(TypedDict):
    preferences: list[str]
    breakfast: list[str]
    lunch: list[str]
    dinner: list[str]

# -------- Init LLM --------
# llm = init_chat_model("google_genai:gemini-2.0-flash")


def get_preferences(state: workflowState):
    state["preferences"] = "vegetarian"
    return {"preferences": state["preferences"]}

def gen_breakfast(state: workflowState):
    # result = llm.invoke(f"Generate {state['preferences']} breakfast options")
    # user_input1 = input("Enter breakfast list: ")
    state["breakfast"] = state.get("breakfast") or ["idly", "upma"]
    # print(f"{[breakfast]}-------------")
    #state["breakfast"] = [item.strip() for item in user_input1.split(",")]
    return {"breakfast": state["breakfast"]}

def gen_lunch(state: workflowState):
    # user_input2 = input("Enter lunch list: ")
    state["lunch"] = state.get("lunch") or ["rice", "dal", "vegetable curry"]
    # state["lunch"] = [item.strip() for item in user_input2.split(",")]
    return {"lunch": state["lunch"]}

def gen_dinner(state: workflowState):
    # user_input3 = input("Enter dinner list: ")
    state["dinner"] = state.get("dinner") or ["chapati", "paneer"]
    # state["dinner"] = [item.strip() for item in user_input3.split(",")]
    return {"dinner": state["dinner"]}

def review_meal(state: workflowState, meal_key: str, next_node: str):
    print(f"\nReview {meal_key}: {state[meal_key]}")
    decision = interrupt(f"Approve {meal_key}? Say 'yes' to approve, or anything else to regenerate.")
    # if decision == "yes":
    #   return Command(goto=next_node)
    # else:
    #     user_input = input(f"Enter {meal_key} list: ")
    #     return Command(goto=f"gen_{meal_key}")
    return state

def review_bf(state: workflowState):
    return review_meal(state, "breakfast", "final_bf")

def review_lunch(state: workflowState):
    return review_meal(state, "lunch", "final_lunch")

def review_dinner(state: workflowState):
    return review_meal(state, "dinner", "final_dinner")

def final_bf(state: workflowState):
    return state

def final_lunch(state: workflowState):
    return state

def final_dinner(state: workflowState):
    return state

def final_review(state: workflowState):
    print("\nFinal meal plan:")
    print("Breakfast:", state["breakfast"])
    print("Lunch:", state["lunch"])
    print("Dinner:", state["dinner"])
    return state

graph = StateGraph(workflowState)
memory = MemorySaver()


graph.add_node("get_preferences", get_preferences)
graph.add_node("gen_breakfast", gen_breakfast)
graph.add_node("review_bf", review_bf)
graph.add_node("final_bf", final_bf)

graph.add_node("gen_lunch", gen_lunch)
graph.add_node("review_lunch", review_lunch)
graph.add_node("final_lunch", final_lunch)

graph.add_node("gen_dinner", gen_dinner)
graph.add_node("review_dinner", review_dinner)
graph.add_node("final_dinner", final_dinner)


graph.add_node("final_review", final_review)

# --------------------------------------
graph.add_edge(START, "get_preferences")
graph.add_edge("get_preferences", "gen_breakfast")
graph.add_edge("get_preferences", "gen_lunch")
graph.add_edge("get_preferences", "gen_dinner")

graph.add_edge("gen_breakfast", "review_bf")
graph.add_edge("review_bf", "final_bf")
graph.add_edge("final_bf", "final_review")

graph.add_edge("gen_lunch", "review_lunch")
graph.add_edge("review_lunch", "final_lunch")
graph.add_edge("final_lunch", "final_review")

graph.add_edge("gen_dinner", "review_dinner")
graph.add_edge("review_dinner", "final_dinner")
graph.add_edge("final_dinner", "final_review")

# End
graph.add_edge("final_review", END)

# -------- Compile--------
workflow = graph.compile(checkpointer=memory)

#------------------------------------------------------
from IPython.display import Image, display

try:
  display(Image(workflow.get_graph().draw_mermaid_png()))
except Exception:
    print(Exception)
    #This requires some extra dependencies and is optional
    pass

#---------------------------------------------------------------
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
result = workflow.invoke({}, config=config, stream_mode="updates")
result



<class 'Exception'>

Review breakfast: ['idly', 'upma']

Review dinner: ['chapati', 'paneer']

Review lunch: ['rice', 'dal', 'vegetable curry']


[{'get_preferences': {'preferences': 'vegetarian'}},
 {'gen_breakfast': {'breakfast': ['idly', 'upma']}},
 {'gen_dinner': {'dinner': ['chapati', 'paneer']}},
 {'gen_lunch': {'lunch': ['rice', 'dal', 'vegetable curry']}},
 {'__interrupt__': (Interrupt(value="Approve breakfast? Say 'yes' to approve, or anything else to regenerate.", resumable=True, ns=['review_bf:8c0b0dcb-ae01-a8d0-73e4-1a06baa2a389']),)},
 {'__interrupt__': (Interrupt(value="Approve dinner? Say 'yes' to approve, or anything else to regenerate.", resumable=True, ns=['review_dinner:e5a88ae1-7e99-19cd-0e50-3ace460bbe6d']),)},
 {'__interrupt__': (Interrupt(value="Approve lunch? Say 'yes' to approve, or anything else to regenerate.", resumable=True, ns=['review_lunch:0b409698-47a7-8c3e-4078-163947b29420']),)}]

In [3]:

def maninloop():
  for chunk in workflow.stream({}, config=config, stream_mode="updates"):
      if "__interrupt__" in chunk:
          # node_name, interrupt_data = chunk["__interrupt__"]
          while True:
            print("\n HUMAN REVIEW REQUIRED:")
            interrupt_obj = chunk["__interrupt__"][0]
            print("Prompt:", interrupt_obj.value)

            decision = input("Approve or Regenerate? (yes to approve / no to regenerate): ").strip().lower()

            if decision == "yes":
              # result = workflow.stream(Command(resume=decision), config=config, stream_mode="updates")
              # print(result)
              prompt = interrupt_obj.value
              if "breakfast" in prompt:
                  result = workflow.invoke(Command(goto="review_bf"), config=config, stream_mode="updates")
                  break
              elif "lunch" in prompt:
                  result = workflow.invoke(Command(goto="review_lunch"), config=config, stream_mode="updates")
                  break
              elif "dinner" in prompt:
                  result = workflow.invoke(Command(goto="review_dinner"), config=config, stream_mode="updates")
                  break
              else:
                  raise ValueError("Could not determine the node")
              print(result)

            else:
              prompt = interrupt_obj.value
              if "breakfast" in prompt:
                  user_input = input("Enter breakfast list: ")
                  #breakfast1 = [item.strip() for item in user_input.split(",")]
                  result = workflow.invoke(Command(goto="gen_breakfast", update={"breakfast": user_input}), config=config, stream_mode="updates")
              elif "lunch" in prompt:
                  user_input = input("Enter lunch list: ")
                  #state["lunch"] = [item.strip() for item in user_input.split(",")]
                  result = workflow.invoke(Command(goto="gen_lunch", update={"lunch": user_input}), config=config, stream_mode="updates")
              elif "dinner" in prompt:
                  user_input = input("Enter dinner list: ")
                  #state["dinner"] = [item.strip() for item in user_input.split(",")]
                  result = workflow.invoke(Command(goto="gen_dinner", update={"dinner": user_input}), config=config, stream_mode="updates")
              else:
                  raise ValueError("Could not determine which meal to regenerate.")
              print(result)

maninloop()
print(result)


Review breakfast: ['idly', 'upma']
Review dinner: ['chapati', 'paneer']

Review lunch: ['rice', 'dal', 'vegetable curry']


 HUMAN REVIEW REQUIRED:
Prompt: Approve dinner? Say 'yes' to approve, or anything else to regenerate.



Review breakfast: ['idly', 'upma']

Review lunch: ['rice', 'dal', 'vegetable curry']

Review dinner: [idly]
[{'__interrupt__': (Interrupt(value="Approve lunch? Say 'yes' to approve, or anything else to regenerate.", resumable=True, ns=['review_lunch:90607a6a-b837-6598-1879-98d951f6587e']),)}, {'__interrupt__': (Interrupt(value="Approve breakfast? Say 'yes' to approve, or anything else to regenerate.", resumable=True, ns=['review_bf:46c0b50d-bca3-3fd5-59ad-45c6cc0bd303']),)}, {'gen_dinner': {'dinner': '[idly]'}}, {'__interrupt__': (Interrupt(value="Approve dinner? Say 'yes' to approve, or anything else to regenerate.", resumable=True, ns=['review_dinner:b22c1dc8-ac5b-2a6d-bb5d-44d35cdc54cf']),)}]

 HUMAN REVIEW REQUIRED:
Prompt: Approve dinner? Say 'yes' to approve, or anything else to regenerate.

Review breakfast: ['idly', 'upma']

Review lunch: ['rice', 'dal', 'vegetable curry']

Review dinner: [idly]

 HUMAN REVIEW REQUIRED:
Prompt: Approve lunch? Say 'yes' to approve, or anything 